Importing relevant packages from PyTorch

In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan 24 16:02:40 2021

@author: s2110992
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

#mnist_data = datasets.MNIST('data',train=True, download = True, transform = transforms.ToTensor())
#mnist_data = list(mnist_data)[:4096] #data of size powers of 2 works better


Simulating Data

In [86]:
training_data_size = 50
data_size = 5
Test_data_uniform = np.random.uniform(low = 0.0, high = 1.0, size = (data_size))
Test_data_normal = np.random.normal(loc = 0.0, scale = 1.0, size = (data_size))
data = []
for i in range(training_data_size):
    data.append(np.random.uniform(low = 0.0, high = 1.0, size = (data_size)))
data_tensor = torch.tensor(data)

Training Data and Training Parameters

In [87]:
#Note we can make BATCH_SIZE smaller than training data set then it 
#does iterative gradient steps.
LEARNING_RATE = 1e-3
NUM_EPOCHS = 50 
BATCH_SIZE = training_data_size
traindata = Test_data_uniform.transpose()
#data_size = 

#dataloader = DataLoader(data,batch_size = BATCH_SIZE, shuffle = True)

In [91]:
# Creating the architecture of the Neural Network

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(data_size,1),
            nn.Tanh(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(1,data_size),
            nn.Tanh(),
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
net = Autoencoder()
print(net)

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=5, out_features=1, bias=True)
    (1): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): Tanh()
  )
)


In [92]:
#using Mean Square Error as the Loss function.
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [93]:
#Training Step!.

for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in data_tensor:
        print(i)
        # get the inputs; data
        inputs = i

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
         #   print('[%d, %5d] loss: %.3f' %
          #        (epoch + 1, i + 1, running_loss / 2000))
           # running_loss = 0.0

print('Finished Training')

tensor([0.7415, 0.9412, 0.5433, 0.8622, 0.1977], dtype=torch.float64)


RuntimeError: expected scalar type Double but found Float

In [ ]:
#Extracting Hidden Layer: Lower Dimensional Representation of our data and plotting.